In [40]:
import csv
import time
import re
import string
import requests
import pytz
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from email.utils import parsedate_tz
from email.utils import mktime_tz
from sqlalchemy import Column, Integer, DateTime, Interval, PickleType, String, select, func, create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [41]:
# Copyright (c) 2018 Sergio Lira <sergio.lira@gmail.com>
#
# MIT Licence. See http://opensource.org/licenses/MIT
#
# Created on 2018-12-03
#

engine = create_engine('sqlite://', echo=False)
Base = declarative_base(bind=engine)

class TimeEntry(Base):
    __tablename__ = 'time_entry'
    id = Column(Integer, primary_key=True)
    date_time = Column(DateTime)
    sdate = Column(String)
    month = Column(Integer)
    day = Column(Integer)
    year = Column(Integer)
    concepts = Column(PickleType)   
        
    def __init__(self, date_time, sdate, month, day, year, concepts):
        self.date_time = date_time
        self.sdate = sdate
        self.month = month
        self.day = day
        self.year = year
        self.concepts = concepts
        
class TimeSpent(Base):
    __tablename__ = 'time_spent'
    id = Column(Integer, primary_key=True)
    sdate = Column(String)
    year = Column(Integer)
    month = Column(Integer)
    day = Column(Integer)
    time_delta = Column(Integer)
    concept = Column(String)
    
    def __init__(self, sdate, month, day, year, concept, time_delta):
        self.sdate = sdate
        self.month = month
        self.day = day
        self.year = year
        self.concept = concept
        self.time_delta = time_delta

Base.metadata.create_all()
Session = sessionmaker(bind=engine)

class TimeTracker:
    
    def __init__(self, content, n_days=15, n_months = 3, censor_concepts = False):
        self._content = content
        self.n_days = n_days
        self.n_months = n_months
        self._censor_concepts = censor_concepts
        self._db = Session()

    def load_content(self):
        """
        Initializes the TimeTracker database 
        If the content is of file type txt it initializes by parsing the file else by reading each row in content.
        returns row count added
        """
        #Set look back date and cut_date from max between n_days and n_months        
        look_back = max(self.n_days//31, self.n_months)
        self._cut_date = datetime.today() - relativedelta(months=look_back)
        self._cut_date = date(self._cut_date.date().year, self._cut_date.date().month, 1)
        
        if self._content.endswith('.txt'):
            return self._load_fromm_file()
        else:
            return self._load_fromm_file()
        
        
    def _extract_values_from_row(self, row):
        #Clean date row, format and extract date fields
        #row[0] = row[0].replace(",","").replace("at","")
        date_time= datetime.fromtimestamp(mktime_tz(parsedate_tz(row[0])))
        date_time = date_time.astimezone(pytz.utc)
        sdate = date_time.strftime("%Y-%m-%d")
        month = int(date_time.strftime('%m'))
        day = int(date_time.strftime('%d'))
        year = int(date_time.strftime('%Y'))
        #Extract question and score
        concepts = row[1].strip().split(';')
        if self._censor_concepts:
            censored_concepts = []
            for concept in concepts:
                concept = ''.join(['*' if i%3==0 else ch for i, ch in enumerate(concept)])
                censored_concepts.append(concept)
            concepts = censored_concepts
        return TimeEntry(date_time, sdate, month, day, year, concepts)
        
    def _load_fromm_file(self):
        with open(self._content, "r") as txtfile:        
            reader = csv.reader(txtfile, delimiter='|', quotechar='"')    
            row_count = 0
            for row in reader:
                #Skip any row that does not matcht he format, including empty rows.
                if len(row) == 2:
                    #Clean date row, format and extract date fields
                    time_entry = self._extract_values_from_row(row)
                    #Only insert date if it will be used
                    if date(time_entry.year, time_entry.month, time_entry.day) >= self._cut_date:
                        #Write to in-memory database
                        self._db.add(time_entry)
                        self._db.commit()
                    row_count += 1
        return row_count
    
    def _calculate_time(self):
        entries = [entry for entry in self._db.query(TimeEntry).order_by(TimeEntry.date_time)]
        if len(entries) > 0 :
            last_entry = entries[0]
            for current_entry in entries[1:]:
                time_delta = current_entry.date_time - last_entry.date_time
                for concept in last_entry.concepts:
                    #print('{} {}: {}'.format(current_entry.sdate, concept, time_delta))
                    time_spent = TimeSpent(current_entry.sdate, 
                                           current_entry.month, 
                                           current_entry.day,
                                           current_entry.year,
                                           concept.strip(),
                                           time_delta.total_seconds())
                    self._db.add(time_spent)
                    self._db.commit()
                last_entry = current_entry

In [42]:
log_file = '/Users/sergio/Dropbox/Reference/Automation/LogTime.txt'
tt = TimeTracker(log_file, censor_concepts = True)
rows_loaded = tt.load_content()
print('Loaded {} time entry rows, inserted {}.'.format(rows_loaded, tt._db.query(TimeEntry.id).count()))
#for entry in tt._db.query(TimeEntry):
   #print(entry.id, entry.date_time, entry.sdate, entry.month, entry.day, entry.year, entry.concepts)
tt._calculate_time()
print('Found {} time spent entries.'.format(tt._db.query(TimeSpent.id).count()))

Loaded 657 time entry rows, inserted 657.
Found 688 time spent entries.


In [43]:
print('Concept totals:')
print(tt._db.query(TimeSpent.concept, 
                   func.sum(TimeSpent.time_delta))\
                       .group_by(TimeSpent.concept)\
                       .order_by(func.sum(TimeSpent.time_delta).desc()).all())

Concept totals:
[('*le*pi*g', 721869), ('*or*in*', 325142), ('*ea*in* -*Fi*ti*n', 207437), ('*el*-C*re', 149824), ('*nt*rn*t *ed*a', 140565), ('*ho*es', 130701), ('*ni*e', 118143), ('*ro*ec*', 110493), ('*om*ut*ng', 92301), ('*am*', 90054), ('*at*ng', 84971), ('*dm*ni*tr*ti*n', 72840), ('*oc*al*zi*g', 43139), ('*ed*ta*in*', 40555), ('*ea*in* -*No*Fi*ti*n', 24446), ('*on*ex* S*it*h', 19255), ('*tu*yi*g', 17095), ('*ov*', 16383), ('*ap*ne*e', 15566), ('*ig*re*', 13900), ('*ov*es*or*Se*ie*', 13705), ('*ud*ob*ok* o* P*dc*st*', 10035), ('*an*a', 9575), ('*el*xi*g', 7611), ('*oc*or* a*d *ea*th', 6712), ('*ho*pi*g', 5407), ('*xe*ci*in*', 2700)]


https://www.digitalocean.com/community/tutorials/data-analysis-and-visualization-with-pandas-and-jupyter-notebook-in-python-3
http://pbpython.com/simple-graphing-pandas.html